In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import train_test_split

/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


## 정밀도/재현율 트레이드 오프

- 사이킷런은 분류 알고리즘은 (이진 분류) 
- 특정 레이블(1,0)에 속하는지 계산하기 위해서 -> 레이블 결정 확률 구하는 것
- 이진 분류시 임계값 조절시에는 binarizer(threshold =값) 조정 가능 구글링 진행해 보기!

### 필수과제1. 임계값 조절하여 분류 테스트 변화 확인해  보기
- 임계값을 3개정도 (원하는 값으로) 잡고 각각의 평가 지표 변화 확인해 보기!

## 정밀도와 재현율은 어떤 상황에서 중요한 지표로 선택될 것인가?
- 재현율이 중요한 경우는 실제 positive 양성 데이터를 Negative 잘못 판단하게 되면 문제가 되는 경우
- 재현율 가장 중요한 사레는 암환자 판단
- 양성 암 환자를 양성이 아닌 음성으로 판단하면 큰일이다.
- 보험사기 (대출사기) 
- 1인 사기건인데, 0으로 판단하면 안 된다.
- 0을 잘못해서 1로 판단하면? - 코스트 비용 많이 난다.

- 반대로 정밀도가 중요한 경우?
- 스팸 메일 판단 할 때 positive 스팸메일을 negative 일반 메일로 분해도 ( 스팸메일을 일반메일로 분류해도 유저입장에서 그래도 한 번 보면 된다.)
- negative 일반메일 스팸메일로 분류하면 큰 이이슈가 도니다. 반대 문제는

## F1 스코어
- 정밀도와 재현율을 결합한 지표 
- 어느 한 쪽으로 치우치지 않는 수치를 나타낼 때 진행
- F1 2 *(precision*recall)/(precision+recall)

In [2]:
def f_1(pr,re):
    return (2*((pr*re)/(pr+re)))

In [3]:
# 머신러닝 결과
# pr 0.9 re 0.1 무엇이 잘 된건가 ?
# pr 0.5 re 0.5 두 개의 결과를 비교했을 때 어떤 것을 성능이 더 좋다고 말할 수 있는가?

f_1(0.5,0.5)

0.5

In [4]:
f_1(0.9,0.1)

0.18000000000000002

### ROC 곡선과 AUC
- 이진분류 예측에서 사용 하는 것
- AUC 스코어는 이진분류의 예측 성능 측정에 중요하게 사용되는 지표
- ROC 커브 ( Receiver Operation Characterstic Curve)
- ROC 커브 FPR( False Positive Rate) 가 변할 때 TPR( True Positive Rate)이 어떻게 변하는지 나타내는 고선
- FPR x축 , TPR Y 축 FPR변화에 따른 TRP의 변화 곡선을 나타낸 것
- TPR은 재현율, 민감도 
- 특이성(speicificilty) - 민감도에 대응 특이성 지표 ( 1- 재현율, 민감도) 

- 민감도 실제값(양성)이 정확히 예측 되어야 하는 수준 ( 질병있으면 질병이 있다고 양성 판정 )
- 특이성 지표(TNR) 실제값 Negative(음성)이 정확히 예측 되어야 하는 수준 ( 질병이 없는 사람은 질병이 없다고 판단하는 것)

- FPR = FP/(FP+TN) = 1-TNR = 1-특이성 

- ROC곡선 자체는 FPR, TPR 변화의 값을 보는 데 이용하고, 분류의 성능 지표로 사용되는 건 ROC 곡선 면적에 기반한 AUC 값
- Area Under Curve AUC
- 밑 면적이 1에 가까울수록 좋은 수치 
- AUC 가 커질려면 어떻게 해야하지? FPR 작은 상태에서 가장 높은 TPR이 나와야 한다. FPR 작은 상태에서 TPR을 얼마나 얻을 수 있나? 

## 조별과제 제주 교통량 데이터 확인 
- MAE 잔차의 절대값 
- 기본 Base LGBM 을 사용 튜닝은 없고 

ditto제주교통량train.csv, 23조_예측끝내조.csv ,(추가진행예정)train_preprocessing15.csv, (추가진행)데이터중급반_21조_GOAT_train.csv.csv, 이쿠조train_final.csv

In [2]:
train = pd.read_csv('이쿠조train_final.csv')

### 기존 raw train.csv

In [47]:
train = pd.read_csv('train.csv')

In [48]:
train

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,20220623,목,17,1,106,지방도1112호선,0,0,60.0,...,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,20220728,목,21,2,103,일반국도11호선,0,0,60.0,...,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,20211010,일,7,2,103,일반국도16호선,0,0,80.0,...,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,20220311,금,13,2,107,태평로,0,0,50.0,...,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,20211005,화,8,2,103,일반국도12호선,0,0,80.0,...,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,TRAIN_4701212,20211104,목,16,1,107,-,0,0,50.0,...,0,대림사거리,33.422145,126.278125,없음,금덕해운,33.420955,126.273750,없음,20.0
4701213,TRAIN_4701213,20220331,목,2,2,107,-,0,0,80.0,...,3,광삼교,33.472505,126.424368,없음,광삼교,33.472525,126.424890,없음,65.0
4701214,TRAIN_4701214,20220613,월,22,2,103,일반국도12호선,0,0,60.0,...,0,고성교차로,33.447183,126.912579,없음,성산교차로,33.444121,126.912948,없음,30.0
4701215,TRAIN_4701215,20211020,수,2,2,103,일반국도95호선,0,0,80.0,...,0,제6광령교,33.443596,126.431817,없음,관광대학입구,33.444996,126.433332,없음,73.0


In [ ]:
train_sp = train.select_dtypes(exclude = 'object')

In [12]:
train_sp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701217 entries, 0 to 4701216
Data columns (total 16 columns):
 #   Column               Dtype  
---  ------               -----  
 0   base_date            int64  
 1   base_hour            int64  
 2   lane_count           int64  
 3   road_rating          int64  
 4   multi_linked         int64  
 5   connect_code         int64  
 6   maximum_speed_limit  float64
 7   vehicle_restricted   float64
 8   weight_restricted    float64
 9   height_restricted    float64
 10  road_type            int64  
 11  start_latitude       float64
 12  start_longitude      float64
 13  end_latitude         float64
 14  end_longitude        float64
 15  target               float64
dtypes: float64(9), int64(7)
memory usage: 573.9 MB


In [34]:
train = pd.read_csv('ditto제주교통량train.csv')

In [35]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326833 entries, 0 to 326832
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   day_of_week            326833 non-null  int64  
 1   base_hour              326833 non-null  int64  
 2   lane_count             326833 non-null  int64  
 3   road_rating            326833 non-null  int64  
 4   road_name              326833 non-null  int64  
 5   maximum_speed_limit    287489 non-null  float64
 6   weight_restricted      326833 non-null  float64
 7   road_type              326833 non-null  int64  
 8   start_node_name        326833 non-null  int64  
 9   start_latitude         326833 non-null  float64
 10  start_longitude        326833 non-null  float64
 11  start_turn_restricted  326833 non-null  int64  
 12  end_node_name          326833 non-null  int64  
 13  end_latitude           326833 non-null  float64
 14  end_longitude          326833 non-nu

In [32]:
train

,day_of_week,base_hour,lane_count,road_rating,road_name,maximum_speed_limit,weight_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target,season
0,1,17,1,106,45,0.6,32400.0,3,344,33.427747,126.662612,0,344,33.427749,126.662335,0,52.0,1
1,1,21,2,103,33,0.6,0.0,0,26,33.500730,126.529107,1,2,33.504811,126.526240,0,30.0,0
2,4,7,2,103,35,1.0,0.0,0,380,33.279145,126.368598,0,155,33.280072,126.362147,0,61.0,3
3,0,13,2,107,56,0.4,0.0,0,43,33.246081,126.567204,0,178,33.245565,126.566228,0,20.0,1
4,6,8,2,103,34,1.0,0.0,0,232,33.462214,126.326551,0,234,33.462677,126.330152,0,38.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326828,5,20,2,103,34,NaN,0.0,0,277,33.492699,126.428241,0,277,33.492539,126.426231,0,40.0,3
326829,2,16,3,106,52,NaN,0.0,0,214,33.249949,126.505664,0,292,33.249583,126.507874,0,32.0,2
326830,5,10,1,103,35,NaN,32400.0,3,341,33.261655,126.452658,0,341,33.261690,126.453102,0,55.0,2
326831,3,10,2,103,34,NaN,0.0,0,269,33.465318,126.908612,0,202,33.470372,126.902793,0,48.0,2


In [36]:
train.columns

Index(['day_of_week', 'base_hour', 'lane_count', 'road_rating', 'road_name',
       'maximum_speed_limit', 'weight_restricted', 'road_type',
       'start_node_name', 'start_latitude', 'start_longitude',
       'start_turn_restricted', 'end_node_name', 'end_latitude',
       'end_longitude', 'end_turn_restricted', 'target', 'season'],
      dtype='object')

In [37]:
train_x = train[['day_of_week', 'base_hour', 'lane_count', 'road_rating', 'road_name',
       'maximum_speed_limit', 'weight_restricted', 'road_type',
       'start_node_name', 'start_latitude', 'start_longitude',
       'start_turn_restricted', 'end_node_name', 'end_latitude',
       'end_longitude', 'end_turn_restricted','season']]

In [38]:
train_y = train['target']

In [46]:
train

,day_of_week,base_hour,lane_count,road_rating,road_name,maximum_speed_limit,weight_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target,season
0,1,17,1,106,45,0.6,32400.0,3,344,33.427747,126.662612,0,344,33.427749,126.662335,0,52.0,1
1,1,21,2,103,33,0.6,0.0,0,26,33.500730,126.529107,1,2,33.504811,126.526240,0,30.0,0
2,4,7,2,103,35,1.0,0.0,0,380,33.279145,126.368598,0,155,33.280072,126.362147,0,61.0,3
3,0,13,2,107,56,0.4,0.0,0,43,33.246081,126.567204,0,178,33.245565,126.566228,0,20.0,1
4,6,8,2,103,34,1.0,0.0,0,232,33.462214,126.326551,0,234,33.462677,126.330152,0,38.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326828,5,20,2,103,34,NaN,0.0,0,277,33.492699,126.428241,0,277,33.492539,126.426231,0,40.0,3
326829,2,16,3,106,52,NaN,0.0,0,214,33.249949,126.505664,0,292,33.249583,126.507874,0,32.0,2
326830,5,10,1,103,35,NaN,32400.0,3,341,33.261655,126.452658,0,341,33.261690,126.453102,0,55.0,2
326831,3,10,2,103,34,NaN,0.0,0,269,33.465318,126.908612,0,202,33.470372,126.902793,0,48.0,2


In [40]:
X_train , X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3, random_state=111)

In [41]:
LR = lgb.LGBMRegressor(random_state=111).fit(X_train, y_train)

In [42]:
pred = LR.predict(X_test)

In [43]:
pred

array([35.3784969 , 50.95828445, 31.65184442, ..., 51.69023127,
       47.28425358, 44.76940387])

In [44]:
from sklearn.metrics import mean_absolute_error

In [45]:
mean_absolute_error(y_test, pred)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

- 기본 베이스로 전처리 없이 mae 4.8이 나왔다. 
- ditto 4.54
- 예측끝내조 5.64
- 이쿠조 4.718
- 런닝머신조 7.5